<pre>
Copyright 2022-2024 Boris Shminke

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
</pre>

In [1]:
# since both Jupyter and `isabelle-client` use `asyncio` we need to
# enable nested event loops. We don't need that when using
# `isabelle-client` from Python scripts outside Jupyter
import nest_asyncio

nest_asyncio.apply()

In [2]:
# first, we start Isabelle server
from isabelle_client import start_isabelle_server

server_info, isabelle_process = start_isabelle_server()

In [3]:
# server info is the same as printed by `isabelle server` command
print(server_info)

server "isabelle" = 127.0.0.1:44559 (password "b755a9e6-1188-4f53-9383-2d85c0fc8026")



In [4]:
# a process object can be used for additional control
type(isabelle_process)

asyncio.subprocess.Process

In [5]:
isabelle_process

<Process 634>

In [6]:
isabelle_process.pid

634

In [7]:
# now we can create a client instance
from isabelle_client import get_isabelle_client

isabelle = get_isabelle_client(server_info)
isabelle

In [8]:
# the server's commands are implemented as client object methods
responses = isabelle.help()
responses

[IsabelleResponse(response_type='OK', response_body='{"isabelle_id":"29f2b8ff84f3","isabelle_name":"Isabelle2024"}', response_length=None),
 IsabelleResponse(response_type='OK', response_body='["cancel","echo","help","purge_theories","session_build","session_start","session_stop","shutdown","use_theories"]', response_length=118)]

In [9]:
# usually, the last server's response contains something interesting
responses[-1]

IsabelleResponse(response_type='OK', response_body='["cancel","echo","help","purge_theories","session_build","session_start","session_stop","shutdown","use_theories"]', response_length=118)

In [10]:
responses[-1].response_body

'["cancel","echo","help","purge_theories","session_build","session_start","session_stop","shutdown","use_theories"]'

In [11]:
# the response body is usually JSON formatted
import json

data = json.loads(responses[-1].response_body)
data

['cancel',
 'echo',
 'help',
 'purge_theories',
 'session_build',
 'session_start',
 'session_stop',
 'shutdown',
 'use_theories']

In [12]:
%%time
# let's do something more meaningful

session_id = isabelle.session_start()
session_id

CPU times: user 7.95 ms, sys: 3.42 ms, total: 11.4 ms
Wall time: 5.84 s


'2fa8548b-b018-45d6-afc1-bfd4322b3f5a'

In [13]:
# let's generate a dummy problem file in Isabelle format
with open("Problem.thy", "w") as problem_file:
    problem_text = """
theory Problem imports Main
begin
lemma "\<forall> x. \<exists> y. x = y"
sledgehammer
oops
end
    """
    problem_file.write(problem_text)

In [14]:
%%time
# now let's ask the server to solve the problem

responses = isabelle.use_theories(
    theories=["Problem"],
    master_dir=".",
    session_id=session_id
)
responses

CPU times: user 6.33 ms, sys: 504 μs, total: 6.83 ms
Wall time: 2.36 s


[IsabelleResponse(response_type='OK', response_body='{"isabelle_id":"29f2b8ff84f3","isabelle_name":"Isabelle2024"}', response_length=None),
 IsabelleResponse(response_type='OK', response_body='{"task":"ec5bb69a-b02e-4c59-ad90-77aa1c8b167c"}', response_length=None),
 IsabelleResponse(response_type='NOTE', response_body='{"percentage":18,"task":"ec5bb69a-b02e-4c59-ad90-77aa1c8b167c","message":"theory Draft.Problem 18%","kind":"writeln","session":"","theory":"Draft.Problem"}', response_length=161),
 IsabelleResponse(response_type='NOTE', response_body='{"percentage":99,"task":"ec5bb69a-b02e-4c59-ad90-77aa1c8b167c","message":"theory Draft.Problem 99%","kind":"writeln","session":"","theory":"Draft.Problem"}', response_length=161),
 IsabelleResponse(response_type='NOTE', response_body='{"percentage":100,"task":"ec5bb69a-b02e-4c59-ad90-77aa1c8b167c","message":"theory Draft.Problem 100%","kind":"writeln","session":"","theory":"Draft.Problem"}', response_length=163),
 IsabelleResponse(response_

In [15]:
# notice that important messages live inside "nodes" object
data = json.loads(responses[-1].response_body)
data

{'ok': True,
 'errors': [],
 'nodes': [{'messages': [{'kind': 'writeln',
     'message': 'Sledgehammering...',
     'pos': {'line': 5,
      'offset': 60,
      'end_offset': 72,
      'file': 'Problem.thy'}},
    {'kind': 'writeln',
     'message': 'verit found a proof...',
     'pos': {'line': 5,
      'offset': 60,
      'end_offset': 72,
      'file': 'Problem.thy'}},
    {'kind': 'writeln',
     'message': 'cvc4 found a proof...',
     'pos': {'line': 5,
      'offset': 60,
      'end_offset': 72,
      'file': 'Problem.thy'}},
    {'kind': 'writeln',
     'message': 'zipperposition found a proof...',
     'pos': {'line': 5,
      'offset': 60,
      'end_offset': 72,
      'file': 'Problem.thy'}},
    {'kind': 'writeln',
     'message': 'zipperposition found a proof...',
     'pos': {'line': 5,
      'offset': 60,
      'end_offset': 72,
      'file': 'Problem.thy'}},
    {'kind': 'writeln',
     'message': 'vampire found a proof...',
     'pos': {'line': 5,
      'offset': 60,
 

In [16]:
# the messages from `sledgehammer` have an easily parsable format
messages = [
    message["message"] for message in data["nodes"][0]["messages"]
]
messages

['Sledgehammering...',
 'verit found a proof...',
 'cvc4 found a proof...',
 'zipperposition found a proof...',
 'zipperposition found a proof...',
 'vampire found a proof...',
 'verit: Try this: by simp (1 ms)',
 'cvc4: Try this: by fastforce (2 ms)',
 'zipperposition: Duplicate proof',
 'zipperposition: Try this: by auto (2 ms)',
 'vampire: Try this: by blast (1 ms)',
 'spass found a proof...',
 'spass: Duplicate proof',
 'Done']

In [17]:
# we can parse `sledgehammer` suggestions using regular expressions
import re

proposed_solutions = [
    re.findall('.*: Try this: (.*) \(\d+\.?\d* m?s\)', message)
    for message in messages
]
proposed_solutions

[[],
 [],
 [],
 [],
 [],
 [],
 ['by simp'],
 ['by fastforce'],
 [],
 ['by auto'],
 ['by blast'],
 [],
 [],
 []]

In [18]:
# in principle, we can value some solutions more
import random

solution = random.choice([
    proposed_solution[0]
    for proposed_solution in proposed_solutions
    if proposed_solution
])
solution

'by auto'

In [19]:
# now let's write down our solution to a file
with open("Solution.thy", "w") as solution_file:
    solution_text = (
        problem_text
        .replace("sledgehammer\noops\n", f"{solution}\n")
        .replace("theory Problem ", "theory Solution ")
    )
    print(solution_text)
    solution_file.write(solution_text)


theory Solution imports Main
begin
lemma "\<forall> x. \<exists> y. x = y"
by auto
end
    


In [20]:
%%time
# and finally let's ask the server to check the solution

responses = isabelle.use_theories(
    theories=["Solution"],
    master_dir=".",
    session_id=session_id
)
responses

CPU times: user 2.04 ms, sys: 1.02 ms, total: 3.06 ms
Wall time: 2.01 s


[IsabelleResponse(response_type='OK', response_body='{"isabelle_id":"29f2b8ff84f3","isabelle_name":"Isabelle2024"}', response_length=None),
 IsabelleResponse(response_type='OK', response_body='{"task":"0759c22f-7dfd-4990-9c25-3ecbf9f2d29c"}', response_length=None),
 IsabelleResponse(response_type='NOTE', response_body='{"percentage":99,"task":"0759c22f-7dfd-4990-9c25-3ecbf9f2d29c","message":"theory Draft.Solution 99%","kind":"writeln","session":"","theory":"Draft.Solution"}', response_length=163),
 IsabelleResponse(response_type='FINISHED', response_body='{"ok":true,"errors":[],"nodes":[{"messages":[{"kind":"writeln","message":"theorem \\\\<forall>x. \\\\<exists>y. x = y","pos":{"line":5,"offset":61,"end_offset":63,"file":"Solution.thy"}}],"exports":[],"status":{"percentage":100,"unprocessed":0,"running":0,"finished":9,"failed":0,"total":9,"consolidated":true,"canceled":false,"ok":true,"warned":0},"theory_name":"Draft.Solution","node_name":"Solution.thy"}],"task":"0759c22f-7dfd-4990-9c

In [21]:
# the response body doesn't contain any errors
json.loads(responses[-1].response_body)

{'ok': True,
 'errors': [],
 'nodes': [{'messages': [{'kind': 'writeln',
     'message': 'theorem \\<forall>x. \\<exists>y. x = y',
     'pos': {'line': 5,
      'offset': 61,
      'end_offset': 63,
      'file': 'Solution.thy'}}],
   'exports': [],
   'status': {'percentage': 100,
    'unprocessed': 0,
    'running': 0,
    'finished': 9,
    'failed': 0,
    'total': 9,
    'consolidated': True,
    'canceled': False,
    'ok': True,
    'warned': 0},
   'theory_name': 'Draft.Solution',
   'node_name': 'Solution.thy'}],
 'task': '0759c22f-7dfd-4990-9c25-3ecbf9f2d29c'}

In [22]:
# we can gracefully finish the session
isabelle.session_stop(session_id)

[IsabelleResponse(response_type='OK', response_body='{"isabelle_id":"29f2b8ff84f3","isabelle_name":"Isabelle2024"}', response_length=None),
 IsabelleResponse(response_type='OK', response_body='{"task":"327a5326-1945-4fb5-8d4e-df762f6eb0ca"}', response_length=None),
 IsabelleResponse(response_type='FINISHED', response_body='{"ok":true,"return_code":0,"task":"327a5326-1945-4fb5-8d4e-df762f6eb0ca"}', response_length=None)]

In [23]:
# and shut the server down
isabelle.shutdown()

[IsabelleResponse(response_type='OK', response_body='{"isabelle_id":"29f2b8ff84f3","isabelle_name":"Isabelle2024"}', response_length=None),
 IsabelleResponse(response_type='OK', response_body='', response_length=None)]